In [1]:
import org.apache.spark.SparkContext

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4042
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1669437208081)
SparkSession available as 'spark'


import org.apache.spark.SparkContext


In [2]:
import org.apache.spark.SparkConf
//上面这俩不用

import org.apache.spark.SparkConf


In [10]:
import org.apache.spark.mllib.linalg.distributed._
//这个包含了IndexedRow

import org.apache.spark.mllib.linalg.distributed._


In [1]:
import org.apache.spark.mllib.linalg.{Vectors,Matrix => sparkMatrix,DenseMatrix => sparkDenseMatrix}

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1671621601213)
SparkSession available as 'spark'


import org.apache.spark.mllib.linalg.{Vectors, Matrix=>sparkMatrix, DenseMatrix=>sparkDenseMatrix}


In [2]:
import breeze.linalg._  //为了给矩阵求逆
import breeze.stats.distributions._
import scala.math._

import breeze.linalg._
import breeze.stats.distributions._
import scala.math._


In [3]:
//要写一个转化函数，把sparkmatrix变成DenseMatrix
def tobreezematrix(tmp1: sparkMatrix):DenseMatrix[Double]={
    var tmp2:DenseMatrix[Double]=DenseMatrix.zeros[Double](tmp1.numRows,tmp1.numCols) //调用行数列数，DenseMatrix里用的是.rows, .cols
    for (i <- 0 until tmp1.numRows){
        for (j <- 0 until tmp1.numCols){
            tmp2(i,j)=tmp1(i,j)
        }
    }
    tmp2
}

tobreezematrix: (tmp1: org.apache.spark.mllib.linalg.Matrix)breeze.linalg.DenseMatrix[Double]


In [46]:
val N=sc.broadcast(10000) //X行数
val r1=sc.broadcast(2)
val P=sc.broadcast(100)  //X列数
val tau=sc.broadcast(0.5)
val iter=sc.broadcast(20)  //控制迭代次数
var indeces=0 until N.value  //行的标签，不能1 to N.value，这样X的维数会多1
var Beta=sc.broadcast(DenseVector.ones[Double](P.value)) 

N: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(454)
r1: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(455)
P: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(456)
tau: org.apache.spark.broadcast.Broadcast[Double] = Broadcast(457)
iter: org.apache.spark.broadcast.Broadcast[Int] = Broadcast(458)
indeces: scala.collection.immutable.Range = Range 0 until 10000
Beta: org.apache.spark.broadcast.Broadcast[breeze.linalg.DenseVector[Double]] = Broadcast(459)


In [47]:
var PallallelIndeces=sc.parallelize(indeces)

PallallelIndeces: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[913] at parallelize at <console>:38


In [48]:
var sample_matrix=PallallelIndeces.map(s => {
    var p=100
    var norm_dist=new Gaussian(0,1)
    var x=new DenseMatrix(rows=1,cols=p,norm_dist.sample(p).toArray) //生成一个行向量（一个样本），即用随机数（记得toArray）去填充一个1*p的向量
    //按课件上的公式生成y
    var y=x*Beta.value+x(0,0)*norm_dist.draw()  //draw直接出来一个数，sample出来一个Vector，要得到数还要把它提出来。现在y是一个1*1的DenseVector
    //下面组装我们要的矩阵
    Array(s.toDouble).union(x.toArray).union(Array(y(0))) //x左一个，右一个
})

sample_matrix: org.apache.spark.rdd.RDD[Array[Double]] = MapPartitionsRDD[914] at map at <console>:40


In [49]:
sample_matrix.collect()(0)(0)

res25: Double = 0.0


In [50]:
sample_matrix.persist()//老师好像说不考这个

res26: org.apache.spark.rdd.RDD[Array[Double]] = MapPartitionsRDD[914] at map at <console>:40


In [51]:
var sample_x=sample_matrix.map(f => IndexedRow(f.take(1)(0).toLong,Vectors.dense(f.drop(1).dropRight(1))))

sample_x: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.distributed.IndexedRow] = MapPartitionsRDD[915] at map at <console>:37


In [52]:
sample_x.collect()

res27: Array[org.apache.spark.mllib.linalg.distributed.IndexedRow] = Array(IndexedRow(0,[1.199364306365418,-0.08500119667656966,0.9676936846406997,-0.22130503105056892,-0.21348874265267043,-1.3781950776844358,-1.3929803477363836,0.3070579635694038,-1.2946522983479913,-0.16169586264574268,-2.228944929639637,-0.4514750581556216,-1.4715652768313905,-0.588345229920394,0.44975758405426314,-0.49256101780705086,-0.8891601772363463,0.23074996887114227,1.5818662357121396,-0.23143068209897671,-1.2877361990978524,0.20075845244924256,-1.169858925638768,1.55210137698115,2.2367434142391813,-1.8083540053804816,1.0237654426409284,-0.16322184827534086,-1.215309718929886,1.6945775337478428,0.5943073292783146,-0.026334433107205447,1.0603280358729386,0.18523525907404045,0.37463607285633305,0.51836988733747...


In [53]:
Array(Array(1,2),Array(2,3)).take(1)(0)(0)

res28: Int = 1


In [54]:
var sample_y=sample_matrix.map(f => IndexedRow(f.take(1)(0).toLong,Vectors.dense(f.drop(P.value+1))))

sample_y: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.distributed.IndexedRow] = MapPartitionsRDD[916] at map at <console>:38


In [55]:
var sample_x_indexedrowmatrix=new IndexedRowMatrix(sample_x)

sample_x_indexedrowmatrix: org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@3eeb010


In [56]:
var sample_y_indexedrowmatrix=new IndexedRowMatrix(sample_y)

sample_y_indexedrowmatrix: org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@1eeeacec


In [57]:
var x=sample_x_indexedrowmatrix.toBlockMatrix(rowsPerBlock=N.value/10,colsPerBlock=P.value/10)

x: org.apache.spark.mllib.linalg.distributed.BlockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@59a715a0


In [58]:
var y=sample_y_indexedrowmatrix.toBlockMatrix(rowsPerBlock=N.value/10,colsPerBlock=1)

y: org.apache.spark.mllib.linalg.distributed.BlockMatrix = org.apache.spark.mllib.linalg.distributed.BlockMatrix@b2de921


In [59]:
x.toLocalMatrix.numRows

res29: Int = 10000


In [60]:
x.toLocalMatrix.numCols

res30: Int = 100


In [61]:
//初始化ADMM算法
var i:Int=0
var diff:Double=1.0
val y_local=y.toLocalMatrix()  //block to local
var beta_hat,beta_old:DenseMatrix[Double]=DenseMatrix.zeros[Double](P.value,1)
var z:DenseMatrix[Double]=DenseMatrix.zeros[Double](N.value,1)
var alpha: DenseMatrix[Double]=DenseMatrix.zeros[Double](N.value,1)
val tmp1=x.transpose.multiply(x).toLocalMatrix() //block to local
var tmp2=tobreezematrix(tmp1) //local to breeze
val beta_fixed=inv(tmp2) //用breezematrix求逆

i: Int = 0
diff: Double = 1.0
y_local: org.apache.spark.mllib.linalg.Matrix =
3.9831436069043
5.202622799005775
-10.102359077879763
10.943935599622959
-11.060195132563095
6.246361770588701
-14.996853116305829
8.652474463269135
-7.322862269811567
3.6086370700285944
-6.340745483904115
-12.538714695622206
-11.554387336873916
7.036042655649631
-5.32390727509339
4.597477449094728
-5.091957027271638
5.055261359985077
1.9715586464931996
10.351830488765044
12.514244337963392
-2.591308265830418
-21.644210609178277
-5.082532228644028
-11.060959937434053
-9.027637731273101
-6.9117851503098695
-3.0629428559436565
-2.5754827011682084
-0.8855946031913255
3.9682332671543854
-1.7278497035622213
-2.8209048233463796
-9.636609839078785
-4.241962229218133
9.54542869168...


In [62]:
sample_y_indexedrowmatrix.multiply(y_local.transpose)

res31: org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix = org.apache.spark.mllib.linalg.distributed.IndexedRowMatrix@37cebb78


In [63]:
while(i < iter.value && diff > pow(10,-6)){
    //更新beta
    beta_old=beta_hat
    var tmp3=(tobreezematrix(y_local)-z).toArray
    var beta_right=x.transpose.toIndexedRowMatrix
    .multiply(new sparkDenseMatrix(N.value,1,alpha.map(s => s/r1.value).toArray))
    .toBlockMatrix(rowsPerBlock=P.value/10,colsPerBlock=1)
    .add(x.transpose.toIndexedRowMatrix.multiply(new sparkDenseMatrix(N.value,1,tmp3)).toBlockMatrix(rowsPerBlock=P.value/10,colsPerBlock=1))
    .toLocalMatrix() //变成local方便tobreeze
    
    beta_hat=beta_fixed*tobreezematrix(beta_right)
    //更新z
    var tmp4=y
    .subtract(x.toIndexedRowMatrix.multiply(new sparkDenseMatrix(P.value,1,beta_hat.toArray)).toBlockMatrix(rowsPerBlock=N.value/10,colsPerBlock=1))
    .toLocalMatrix()  //变成local方便tobreeze
    
    z=(tobreezematrix(tmp4)+alpha.map(s => s/r1.value)).map(s => {
        if((tau.value/r1.value) < s) s-tau.value/r1.value
        else if (s<((tau.value-1)/r1.value)) s-(tau.value-1)/r1.value
        else 0
    })
    //更新alpha
    alpha=alpha+(tobreezematrix(tmp4)-z).map(s => s*r1.value)
    diff=sum((beta_old-beta_hat).map(s => abs(s)))//scala.math包里的abs只能对单个数做运算
    println(beta_old-beta_hat)
    i += 1
}

-0.9854504082981331  
-0.9920764002420118  
-0.9831190945858371  
-1.0206884581355482  
-0.9980106318454826  
-1.001809516512235   
-1.0051546041619135  
-1.0015398112376792  
-0.991763499494419   
-0.9982436341977301  
-0.9948745773895169  
-0.9989865944267916  
-0.9962246485314697  
-1.0030681489049211  
-1.0051922193980354  
-0.9907368278044372  
-1.0088963053307198  
-0.9963872413061429  
-1.0154641660162758  
-0.9699846097903273  
-0.9962587148121107  
-0.9926429686957486  
-0.9903720927796141  
-1.003487035361104   
-0.995728932401456   
-1.0088225854315271  
-0.9837449298392872  
-0.98269721941454    
-1.0133531787500236  
-1.0097031669983263  
-0.9887939917707096  
-1.001080416067857   
-0.975302662669892   
-1.0138593193913399  
-1.0021738725990081  
-0.9901907549910463  
-0.998569340970077   
-0.9916215677706454  
-1.0085741504959478  
-0.9769813808838337  
-0.9964523479280039  
-1.000203966848038   
-0.999267349901023   
-0.9999617880367645  
-0.9829480548141522  
-0.9865818

In [64]:
sum(beta_hat)

res33: Double = 100.01717096750156


In [67]:
var old=DenseVector(1.0,2.0,3.0)
var mynew=DenseVector(1.0,2.0,3.0)
old=mynew
mynew=DenseVector(3.0,2.0,3.0)
old

old: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
mynew: breeze.linalg.DenseVector[Double] = DenseVector(3.0, 2.0, 3.0)
old: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)
mynew: breeze.linalg.DenseVector[Double] = DenseVector(3.0, 2.0, 3.0)
res36: breeze.linalg.DenseVector[Double] = DenseVector(1.0, 2.0, 3.0)


In [73]:
var Oldbeta=DenseVector.zeros[Double](3)
var betahat=DenseVector.zeros[Double](3)
Oldbeta(0 until Oldbeta.size):=betahat(0 until betahat.size)
betahat(0)=3.0
Oldbeta

Oldbeta: breeze.linalg.DenseVector[Double] = DenseVector(0.0, 0.0, 0.0)
betahat: breeze.linalg.DenseVector[Double] = DenseVector(3.0, 0.0, 0.0)
res42: breeze.linalg.DenseVector[Double] = DenseVector(0.0, 0.0, 0.0)
